In [1]:
import pandas as pd
from bs4 import BeautifulSoup as soup
import csv
from pathlib import Path

import datetime

now = datetime.datetime.now()

In [2]:
import selenium
from selenium import webdriver

In [3]:
# konfigurasi chromedriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--window-size=1420,1080')
chrome_options.add_argument('--headless')
chrome_options.add_argument('--disable-gpu')

browser = webdriver.Chrome(chrome_options=chrome_options)

/opt/tljh/user/lib/python3.6/site-packages/ipykernel_launcher.py:8: DeprecationWarning: use options instead of chrome_options
  


In [4]:
data_folder = Path("data/")
file_data = 'data_kamar_summary-' + now.strftime("%Y%m%d") + '-' + now.strftime("%H%M") + '.csv'
filename = data_folder / file_data
csvFile = open(filename, 'a')
#Use csv Writer
csvWriter = csv.writer(csvFile)
csvWriter.writerow(['satker', 'nama', 'alamat', 'prov','total_kamar', 'terisi_lk', 'terisi_pr', 'total_terisi','kosong_lk', 'kosong_pr', 'total_kosong', 'waiting_list','lat','lng'])

124

In [5]:
faskes_df = pd.read_csv("Faskes - Rumah Sakit.csv",delimiter=',',encoding='ISO-8859-1')
faskes_df.head()

,Unnamed: 0,prov_id,kode_rs,nama_unit,alamat,nama_prov,jenis_faskes,lat,lng
0,1,12,1277015,RS TNI Tk. IV 01.07.03,Jl. Sudirman No.1 Kelurahan,SUMATERA UTARA,Rumah Sakit,3.596056,98.675812
1,2,12,1277011,RS Umum Daerah Padang Sidempuan,"Jl Dr. F.L. Tobing No. 10,Padangsidimpuan",SUMATERA UTARA,Rumah Sakit,1.419353,99.185211
2,3,12,1276040,RS Tk IV 01.07.02 Binjai,Jl. Bandung No.4 Binjai,SUMATERA UTARA,Rumah Sakit,3.699319,98.507103
3,4,12,1276025,RS Bangkatan PTPN II,Jl. Hasanuddin No.40 Binjai,SUMATERA UTARA,Rumah Sakit,3.600425,98.485710
4,5,12,1275996,RS Umum Sufina Azis,"Jl Karya Baru No.1,Medan",SUMATERA UTARA,Rumah Sakit,3.590356,98.711342


In [6]:
faskes_df.shape

(1222, 9)

In [7]:

for index, row in faskes_df.iterrows():
    satker = str(row['kode_rs'])
    nama_rs = str(row['nama_unit'])
    alamat =  str(row['alamat'])
    prov =  str(row['nama_prov'])
    lat =  row['lat']
    long =  row['lng']
    
    link = 'http://sirs.yankes.kemkes.go.id/integrasi/data/bed_monitor.php?satker='+satker
    browser.get(link)
    #print(r)
    data = browser.page_source
    url = soup(data,"lxml")
    

    table = url.find('table', attrs={'class':'tbl-responsive table table-striped table-bordered'})
    #print(table)

    if table is not None:
        res = []
        table_rows = table.find_all('tr')

        num_rows = len(table_rows)
        #print(satker+'-'+nama_rs)

        i = 0
        for tr in table_rows:
            _satker = satker
            _nama = nama_rs
            _alamat = alamat
            _prov = prov
            _lat = lat
            _long = long
            _total_kamar = '-'
            _terisi_lk = '-'
            _terisi_pr = '-'
            _total_terisi = '-'
            _kosong_lk = '-'
            _kosong_pr = '-'
            _total_kosong = '-'
            _waiting_list = '-'
            

            if i==num_rows-1:
                td = tr.find_all('td')
                row = [tr.text.strip() for tr in td if tr.text.strip()]
                
                if row:
                    _total_kamar = row[1] 
                    _terisi_lk = row[2] 
                    _terisi_pr = row[3]
                    _total_terisi = row[4]
                    _kosong_lk = row[5]
                    _kosong_pr = row[6]
                    _total_kosong = row[7]
                    _waiting_list = row[8]

                    #print(_waiting_list)
                    
                csvWriter.writerow([_satker,  _nama, _alamat, _prov, _total_kamar, _terisi_lk, _terisi_pr, _total_terisi,  _kosong_lk, _kosong_pr, _total_kosong, _waiting_list, _lat, _long])
            i = i +1


In [8]:
csvFile.close()

In [9]:
dataset = pd.read_csv(filename)
dataset.shape

(611, 14)

In [10]:
dataset.head(100)

,satker,nama,alamat,prov,total_kamar,terisi_lk,terisi_pr,total_terisi,kosong_lk,kosong_pr,total_kosong,waiting_list,lat,lng
0,1277011,RS Umum Daerah Padang Sidempuan,"Jl Dr. F.L. Tobing No. 10,Padangsidimpuan",SUMATERA UTARA,159,0,0,0,0,0,159,0,1.419353,99.185211
1,1276040,RS Tk IV 01.07.02 Binjai,Jl. Bandung No.4 Binjai,SUMATERA UTARA,64,12,11,23,4,2,41,0,3.699319,98.507103
2,1275794,RS Umum Haji Medan,Jl RS Haji Medan Estate,SUMATERA UTARA,251,29,24,53,107,84,198,0,3.578550,98.688805
3,1275655,RSUP H. Adam Malik,"Jl Bunga Lau No. 17, Medan",SUMATERA UTARA,788,151,99,250,0,0,538,0,3.518900,98.610298
4,1275633,RS Umum Martha Friska,"Jl KL. Yos Sudarso Km.6 No. 91,Medan",SUMATERA UTARA,4,3,0,3,1,0,1,0,3.683069,98.665321
5,1275622,RS Umum Imelda Pekerja Indonesia,"Jl Bilal No.24 Pulo Brayan Darat I,Medan",SUMATERA UTARA,0,0,0,0,0,5,0,0,3.622750,98.683945
6,1275600,RS Umum Prima Husada Cipta Medan,"Jl Stasiun No.92, Belawan Medan",SUMATERA UTARA,83,34,23,57,10,16,26,0,3.772269,98.681465
7,1275590,RS Umum Bhakti,Jl HM Joni No.64 Medan,SUMATERA UTARA,0,0,0,0,1,1,0,0,3.568767,98.697014
8,1275276,RS Umum Vina Estetica,Jl Sultan Iskandar Muda No.119,SUMATERA UTARA,64,0,0,0,2,2,64,0,3.587061,98.661293
9,1275221,RS Jiwa Prof. Dr. Muhammad Ildrem,Jl Let. Jend. Jamin Ginting S Km. 10/ Jl. Tali...,SUMATERA UTARA,441,235,64,299,68,74,142,0,3.585222,98.675575
